In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq


In [11]:
country_list = pd.read_csv('./data/CensusCountryMapping_FRBAtl_short.csv', dtype={'CTY_CODE': str})

country_list.loc[len(country_list)] = ["0003", "EUE", "EUROPEAN UNION"]

country_list.loc[len(country_list)] = ["5082", "GZA", "Gaza"]

country_list.loc[len(country_list)] = ["5083", "WBK", "West Bank"]

country_list = country_list.dropna()

country_list["tariff"] = 10.0

country_list.loc[country_list["CTY_CODE"] == "5700", "tariff"] = 30.0

country_list.loc[country_list["CTY_CODE"] == "5820", "tariff"] = 30.0

country_list.loc[country_list["CTY_CODE"] == "1220", "tariff"] = ( 8.75 / 25.0 ) * 35.0

country_list.loc[country_list["CTY_CODE"] == "2010", "tariff"] = 4.5

country_list.loc[country_list["CTY_CODE"] == "0003", "tariff"] = 10.0

recipricol = pd.read_csv('./tariff-lists/reciprocal-tariffs-annex-I.csv', dtype={'CTY_CODE': str})

recipricol.rename(columns={"tariff_rate": "rtariff"}, inplace = True)

country_list = pd.merge(left = country_list, right = recipricol[["rtariff", "CTY_CODE"]], on = "CTY_CODE", how = "left")

country_list["rtariff"] = country_list["rtariff"].fillna(10.0)

country_list.loc[country_list["CTY_CODE"] == "5700", "rtariff"] = 34.0

country_list.loc[country_list["CTY_CODE"] == "5820", "rtariff"] = 34.0
#IF EU you get 50 and you'll like it

country_list["country"] = country_list["country"].str.upper()


In [12]:
country_list.head(50)

,CTY_CODE,ISO3,country,tariff,rtariff
0,4271,AND,ANDORRA,10.00,10.00
1,5200,ARE,UNITED ARAB EMIRATES,10.00,10.00
2,5310,AFG,AFGHANISTAN,10.00,10.00
3,2484,ATG,ANTIGUA AND BARBUDA,10.00,10.00
4,2481,AIA,ANGUILLA,10.00,10.00
5,4810,ALB,ALBANIA,10.00,10.00
6,4631,ARM,ARMENIA,10.00,10.00
7,7620,AGO,ANGOLA,10.00,32.00
8,3570,ARG,ARGENTINA,10.00,10.00
9,4330,AUT,AUSTRIA,10.00,10.00


In [13]:
new_list = pd.read_csv('./tariff-lists/clean-august-7-tariffs.csv')

new_list.rename({"country_name":"country"}, axis = 1, inplace = True)

new_list["country"] = new_list["country"].str.upper()

country_list = pd.merge(country_list, new_list, on="country", how="left")

country_list["rtariff-aug"] = country_list["rtariff-aug"].fillna(country_list["rtariff"])

country_list.loc[country_list["country"] == "EUROPEAN UNION", "rtariff-aug"] = 15.0

country_list.loc[country_list["country"] == "KOREA, SOUTH", "rtariff-aug"] = 15.0

country_list.loc[country_list["country"] == "INDIA", "rtariff-aug"] = 50.0

country_list.loc[country_list["country"] == "BRAZIL", "rtariff-aug"] = 50.0


In [14]:
eu_iso3_codes = [
    "AUT", "BEL", "BGR", "HRV", "CYP", "CZE", "DNK", "EST", "FIN",
    "FRA", "DEU", "GRC", "HUN", "IRL", "ITA", "LVA", "LTU", "LUX",
    "MLT", "NLD", "POL", "PRT", "ROU", "SVK", "SVN", "ESP", "SWE"
]

country_list.loc[country_list["ISO3"].isin(eu_iso3_codes), "rtariff-aug"] = 15.0

In [17]:
country_list.loc[country_list["country"] == "BRAZIL"]

,CTY_CODE,ISO3,country,tariff,rtariff,rtariff-aug
25,3510,BRA,BRAZIL,10.0,10.0,50.0


In [18]:
expemption_list = pd.read_csv('./tariff-lists/annex-II-exemptions.csv', dtype={'HTSUS': str})
# these are the annex II exemptions from the reciprical tariff list

expemption_list_phones = pd.read_csv('./tariff-lists/41225_exemptions.csv', dtype={'HTSUS': str})
# these are the phone exemptions

steel_list = pd.read_csv('./tariff-lists/steel-tariffs.csv', dtype={'HTSUS': str})
# these are the steel products subject to the new 232 tariffs. This is at 8 digit level.

alu_list = pd.read_csv('./tariff-lists/alu-tariffs.csv', dtype={'HTSUS': str})
# these are the aluminum products subject to the new 232 tariffs. This is at a mixed 8 and 10 digit level.

alu_8_list = alu_list[alu_list['HTSUS'].str.len() == 8]

auto_list = pd.read_csv('./tariff-lists/auto-tariffs.csv', dtype={'HTSUS': str})
# these are the aluminum products subject to the new 232 tariffs. This is at a mixed 8 and 10 digit level.

auto_6_list = auto_list[auto_list['HTSUS'].str.len() == 6]

auto_8_list = auto_list[auto_list['HTSUS'].str.len() == 8]

copper_list = pd.read_csv('./tariff-lists/copper-list.csv', dtype={'HTSUS': str})

brazil_nonaircraft_exemptions = pd.read_csv('./tariff-lists/brazil-non-airplane-exemptions.csv', dtype={'HTSUS': str})

brazil_aircraft_exemptions = pd.read_csv('./tariff-lists/brazil-airplane-exemptions.csv', dtype={'HTSUS': str})

# country_list = pd.read_csv('./tariff-lists/reciprocal-tariffs-annex-I.csv',dtype={'CTY_CODE': str})

In [18]:
def weighted_average(df, value_col, weight_col):
    """
    Compute the weighted average of `value_col` using `weight_col` as weights.
    """
    return (df[value_col] * df[weight_col]).sum() / df[weight_col].sum()

In [19]:
def get_all_imports(cnty_code, tariff, ISO3):

    my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"

    hs = 'HS10'

    end_use = "hs?get=CTY_NAME,CON_VAL_YR,CAL_DUT_YR,I_COMMODITY,I_COMMODITY_SDESC"

    surl = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 

    surl  = surl + my_key + "&time=" + "2024-12" + "&COMM_LVL=" + hs 

    url = surl + "&CTY_CODE=" + cnty_code

    r = requests.get(url) 
    
    print(r)
    
    df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    df.columns = r.json()[0]

    #######################################################################

    df.time = pd.to_datetime(df.time, format="%Y-%m")

    df["imports"] = df["CON_VAL_YR"].astype(float)

    df["duty"] = df["CAL_DUT_YR"].astype(float)

    df["share"] = df["imports"] / df["imports"].sum()
            
    df[hs] = df["I_COMMODITY"].astype(str)

    df["HS8"] = df["HS10"].str[0:8]

    df["HS6"] = df["HS10"].str[0:6]

    df["HS2"] = df["HS10"].str[0:2]

    #######################################################################
    # this then filters so products are only those that are subject to the new tariffs
    # the reciprocal exemptions, steel, aluminum and auto tariffs.

    df["tariff"] = tariff

    exemption = df["HS10"].isin(expemption_list["HTSUS"].tolist()) 

    df.loc[exemption, "tariff"]  = 0.0

    ####################################################
    # this is the phone exemption list. This is a 10 digit list.

    exemption_phones = df["HS10"].isin(expemption_list_phones["HTSUS"].tolist()) 

    if cnty_code == "5700": # if china then still Fentanyl 20 percent on the phones

        df.loc[exemption_phones, "tariff"] = 20.0

    else:

        df.loc[exemption_phones, "tariff"] = 0.0

    ####################################################

    steel = df["HS8"].isin(steel_list["HTSUS"].tolist())

    if cnty_code == "4120": # if the UK, then exempted from the increase

        df.loc[steel, "tariff"]  = 25.0 

    else:

        df.loc[steel, "tariff"]  = 50.0 


    ####################################################

    alu = df["HS10"].isin(alu_list["HTSUS"].tolist()) | df["HS8"].isin(alu_8_list["HTSUS"].tolist())

    if cnty_code == "4120": # if the UK, then exempted from the increase
    
        df.loc[alu, "tariff"]  = 25.0 

    else:

        df.loc[alu, "tariff"]  = 50.0
        

    ####################################################


    auto = df["HS8"].isin(auto_8_list["HTSUS"].tolist()) | df["HS10"].isin(auto_list["HTSUS"].tolist()) | df["HS6"].isin(auto_6_list["HTSUS"].tolist())

    if cnty_code == "1220": # if canada or mexico, then assumption is no tariff

        df.loc[auto, "tariff"]  = 0.0

    elif cnty_code == "2010":

        df.loc[auto, "tariff"]  = 0.0

    elif cnty_code == "5880":
    # Japan gets an auto deal

        df.loc[auto, "tariff"]  = 15.0

    elif cnty_code == "0003":
    #EU gets and auto deal

        df.loc[auto, "tariff"]  = 15.0

    elif ISO3 in eu_iso3_codes:

        df.loc[auto, "tariff"]  = 15.0

    elif cnty_code == "5800":
        #Korea gets an auto deal

        df.loc[auto, "tariff"]  = 15.0

    else:
        df.loc[auto, "tariff"]  = 25.0


    ####################################################
            
    copper = df["HS8"].isin(copper_list["HTSUS"].tolist())

    df.loc[copper, "tariff"]  = 50.0

    ####################################################

    brazil_one = df["HS8"].isin(brazil_nonaircraft_exemptions["HTSUS"].tolist())
    
    brazil_two = df["HS8"].isin(brazil_aircraft_exemptions["HTSUS"].tolist())

    if cnty_code == "3510": # Brazil

        df.loc[brazil_one, "tariff"]  = 0.0

        df.loc[brazil_two, "tariff"]  = 0.0


    return df

In [20]:
for index, row in country_list.iterrows():

    print(f"Processing {row['country']}")

    out_file = f'./data-hs2/hs2-tariff-{row["CTY_CODE"]}.csv'

    # if os.path.exists(out_file):
        
    #     print("Already have downloaded file")
        
    #     continue
    
    df = get_all_imports(row["CTY_CODE"], row["rtariff-aug"], row["ISO3"])

    dfhs2 = df.groupby("HS2").agg({"imports": "sum", "duty": "sum", "CTY_NAME": "first", 
                              "CTY_CODE": "first"}).reset_index()

    foo = df.groupby("HS2").apply(lambda x: weighted_average(x, "tariff", "imports"))

    dfhs2 = dfhs2.merge(foo.rename("2025_tariff_increase"), left_on="HS2", right_index=True)

    dfhs2.rename(columns={"imports": "2024_imports", "duty": "2024_duty"}, inplace=True)

    dfhs2["2024_tariff"] = 100* ( dfhs2["2024_duty"] / dfhs2["2024_imports"] )

    dfhs2.to_csv(out_file, index=False)

Processing ANDORRA
<Response [200]>


NameError: name 'weighted_average' is not defined

In [44]:
checkdf = pd.read_csv(f'./data-hs2/hs2-tariff-1220.csv')

In [45]:
(checkdf["2025_tariff_increase"] * checkdf["2024_imports"]).sum() / checkdf["2024_imports"].sum()

6.676695201750996

In [46]:
checkdf["2024_duty"].sum() / checkdf["2024_imports"].sum()

0.001004719282005802

In [47]:
checkdf[checkdf["HS2"] == 74]

,HS2,2024_imports,2024_duty,CTY_NAME,CTY_CODE,2025_tariff_increase,2024_tariff
73,74,3.994336e+09,761757.0,CANADA,1220,28.034191,0.019071


In [30]:
checkdf["2025_tariff_increase"]

0     10.0
1     10.0
2     10.0
3     10.0
4     10.0
      ... 
86    10.0
87    10.0
88    10.0
89    10.0
90    10.0
Name: 2025_tariff_increase, Length: 91, dtype: float64